<a href="https://colab.research.google.com/github/lizardnote/NLP/blob/main/%EA%B9%80%ED%9A%A8%EC%9E%AC_1%EC%B0%A8_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필수 라이브러리 설치
!pip install -U -q datasets transformers fsspec
!pip install -q sentence-transformers faiss-cpu matplotlib seaborn pandas numpy tqdm scikit-learn nltk python-Levenshtein huggingface_hub
!pip install numpy==1.23.5


ERROR: unknown command "intall" - maybe you meant "install"
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_sys

In [ ]:
# 2. 허깅페이스 로그인
from huggingface_hub import login
# 토큰 문자열 직접 입력
login("인증토큰")

In [ ]:
from datasets import load_dataset
import pandas as pd
import numpy as np

# 3-1. 데이터 로드
dataset = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset", split="train")

df1 = dataset.to_pandas()
# print(df1.columns)  # 실제 컬럼명 확인
# null 제거
df1 = df1[['instruction', 'response']].dropna()
df1.head(3)

# 3-2. 데이터 로드
dataset = load_dataset("argilla/customer_assistant", split = 'train')
df2 = dataset.to_pandas()
# null 제거
df2 = df2[['context', 'user-message', 'response-suggestion']].dropna()
# context와 user-message 합치기 (구분자 추가)
df2['instruction'] = df2['context'] + "\n\n" + df2['user-message']
df2['response'] = df2['response-suggestion']
df2 = df2[['instruction', 'response']]
df2.head(3)

# 3-3. 데이터 로드
dataset = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset", split = 'train')
df3 = dataset.to_pandas()
# null 제거
df3 = df3[['instruction', 'response']].dropna()
df3.head(3)

# sampling 100개씩
df1_ = df1.iloc[:100]
df2_ = df2.iloc[:100]
df3_ = df3.iloc[:100]
df_all = pd.concat([df1_, df2_, df3_], ignore_index=True)


In [ ]:
from sklearn.model_selection import train_test_split
# 4. train test split
train_df, test_df = train_test_split(df_all, test_size=0.2, random_state=42)

from tqdm import tqdm
tqdm.pandas()

# 5. 템플릿 변환
for split_df in [train_df, test_df]:
    split_df['messages'] = split_df.progress_apply(lambda x: [
        {"role": "user", "content": x['instruction']},
        {"role": "assistant", "content": x['response']}
    ], axis=1)

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

for split_df in [train_df, test_df]:
    split_df['text'] = split_df['messages'].progress_apply(
        lambda x: tokenizer.apply_chat_template(
            x, tokenize=False, add_generation_prompt=False
        )
    )

# 변환 후 저장
train_df[['text']].to_json("train_dataset.jsonl", orient="records", lines=True)
test_df[['text']].to_json("test_dataset.jsonl", orient="records", lines=True)

100%|██████████| 60/60 [00:00<00:00, 7480.70it/s]


In [ ]:
from google.colab import files
files.download("train_dataset.jsonl")
files.download("test_dataset.jsonl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>